In [ ]:
# Demo — Multi-Asset Trading (SMA backtest + Plotly)
This notebook demonstrates:
- adding the repo root to `sys.path` so `src/` imports work,
- loading `data/sample_multi_asset.csv` with `MarketDataLoader`,
- running a simple SMA-based backtest (same logic as `src/cli.py`),
- plotting price series and equity curve inline with Plotly using `src.utils`.

Run the cells in order.


In [ ]:
# Ensure repo root is on sys.path so we can import from src/
import sys, os
nb_dir = os.getcwd()
repo_root = os.path.abspath(os.path.join(nb_dir, "..")) if os.path.basename(nb_dir) == "notebooks" else nb_dir
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("repo_root:", repo_root)


In [ ]:
# Standard packages and friendly display options
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# We will use Plotly for interactive inline plots
import plotly.io as pio
pio.renderers.default = "notebook"  # or "notebook_connected" if available


In [ ]:
# Import classes and functions from the new src/ package
from src.data.loader import MarketDataLoader
from src.strategies.sma import simple_moving_average_signals
from src.backtest.vector_backtester import vector_backtest
from src.utils import plot_price_timeseries, plot_equity_curve


In [ ]:
# Path to the sample data (relative to repo root)
CSV_PATH = "../data/sample_multi_asset.csv"  # If notebook directory is /notebooks
# If you run notebook from repo root, set CSV_PATH = "data/sample_multi_asset.csv"

loader = MarketDataLoader(CSV_PATH)
df = loader.load()
dates, asset_names, prices = loader.price_matrix(df)

print("Loaded data:", df.shape)
print("Assets:", asset_names[:10])
df.head()


In [ ]:
# Pick an asset to demo (first asset)
asset_idx = 0
asset_name = asset_names[asset_idx]
pseries = prices[:, asset_idx]

# SMA parameters (same defaults used by CLI)
short_w, long_w = 20, 50
signals = simple_moving_average_signals(pseries, window_short=short_w, window_long=long_w)  # length T

# Align returns: compute simple returns (period t uses price[t] vs price[t-1])
rets = np.concatenate(([0.0], (pseries[1:] / pseries[:-1] - 1.0)))  # length T

# Backtest expects 2D arrays (T,N)
signals_mat = signals.reshape(-1, 1)
returns_mat = rets.reshape(-1, 1)

equity, total_return, sharpe = vector_backtest(signals_mat, returns_mat, transaction_cost=0.0005)

print(f"Asset: {asset_name}")
print(f"Total return: {total_return:.4%}, Sharpe (ann): {sharpe:.2f}")


In [ ]:
# Plot full set of asset price series (all assets)
fig_prices = plot_price_timeseries(dates, prices, asset_names, title="Prices — All Assets")
fig_prices.show()

# Plot equity curve for the demo strategy
fig_equity = plot_equity_curve(dates, equity, title=f"Equity Curve — {asset_name} SMA({short_w},{long_w})")
fig_equity.show()


In [ ]:
## Suggested experiments
- Change `asset_idx` to another asset and re-run the SMA backtest.
- Modify SMA windows (`short_w`, `long_w`) and observe the equity curve.
- Replace SMA with `momentum_signals` (from `src/strategies/momentum.py`) and compare.
- Run cross-sectional portfolio: create signals for each asset and backtest (vectorized).
